In [ ]:
import dask_awkward as dak
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import dask_awkward as dak
import numpy as np
import json
import argparse
import os
from distributed import Client
import time    
import tqdm
import mplhep as hep
import matplotlib.pyplot as plt
import matplotlib
import glob
hep.style.use("CMS")
from dask_gateway import Gateway
# gateway = Gateway(
#     "http://dask-gateway-k8s.geddes.rcac.purdue.edu/",
#     proxy_address="traefik-dask-gateway-k8s.cms.geddes.rcac.purdue.edu:8786",
# )
# cluster_info = gateway.list_clusters()[0]# get the first cluster by default. There only should be one anyways
# client = gateway.connect(cluster_info.name).get_client()
# print("Gateway Client created")
client =  Client(n_workers=30,  threads_per_worker=2, processes=True, memory_limit='10 GiB') 

In [2]:
# get data events
# common_load_path = "/depot/cms/users/yun79/results/stage1/Run2StorageTest/2018/f1_0"
common_load_path = "/work/users/yun79/stage1_output/Run2StorageTest/2018/f1_0"
data_load_path = common_load_path+"/data*/*/*.parquet"
# data_load_path = common_load_path+"/data_C/*/*.parquet"
# data_load_path = common_load_path+"/data_D/*/*.parquet"

data_events = dak.from_parquet(data_load_path) 
# we're only interested in ZCR
filter = ak.fill_none(data_events["z_peak"], value=False)
data_events = data_events[filter]



In [3]:
! ls /work/users/yun79/stage1_output/Run2StorageTest/2018/f1_0

data_A		       outputRun2Storage_dataB_redoNDym50.log  ttjets_dl
data_B		       outputRun2Storage_data.log	       ttjets_sl
data_C		       outputRun2Storage_DYnTT_dl.log	       vbf_powheg
data_D		       outputRun2Storage.log		       ww_2l2nu
dy_M-100To200	       outputRun2Storage_restOfVVnSignal.log   wz_1l1nu2q
dy_M-50		       outputRun2Storage_TTDLnMostVV.log       wz_2l2q
ewk_lljj_mll50_mjj120  st_tw_antitop			       wz_3lnu
ggh_powheg	       st_tw_top			       zz


In [4]:
# get MC events (all MC except DY)
secondary_MC_strs = [
    "ttjet*", # all TT samples
    "st_tw*", # single Top samples
    "ewk*", # single Top samples
    "ww*", # diboson samples
    "wz*", # diboson samples
    "zz*", # diboson samples
]
filelist_big = []
for mc_str in secondary_MC_strs:
    filelist = glob.glob(common_load_path + f"/{mc_str}/*/*.parquet")
    filelist_big += filelist
mc_events = dak.from_parquet(filelist_big) 
# we're only interested in ZCR
filter = ak.fill_none(mc_events["z_peak"], value=False)

mc_events = mc_events[filter]
# get dy MC events

dy_load_path = common_load_path+"/dy_*/*/*.parquet"
dy_events = dak.from_parquet(dy_load_path) 
# we're only interested in ZCR
filter = ak.fill_none(dy_events["z_peak"], value=False)
dy_events = dy_events[filter]

In [5]:
def get_calib_categories(events):
    BB = ((abs(events["mu1_eta"])<=0.9) & (abs(events["mu2_eta"])<=0.9))
    BO = ((abs(events["mu1_eta"])<=0.9) & ((abs(events["mu2_eta"])>0.9) & (abs(events["mu2_eta"]) <=1.8)))
    BE = ((abs(events["mu1_eta"])<=0.9) & ((abs(events["mu2_eta"])>1.8) & (abs(events["mu2_eta"]) <=2.4)))
    OB = (((abs(events["mu1_eta"])>0.9) & (abs(events["mu1_eta"]) <=1.8)) & (abs(events["mu2_eta"])<=0.9))
    OO = (((abs(events["mu1_eta"])>0.9) & (abs(events["mu1_eta"]) <=1.8)) & ((abs(events["mu2_eta"])>0.9) & (abs(events["mu2_eta"]) <=1.8)))
    OE = (((abs(events["mu1_eta"])>0.9) & (abs(events["mu1_eta"]) <=1.8)) & ((abs(events["mu2_eta"])>1.8) & (abs(events["mu2_eta"]) <=2.4)))
    EB = (((abs(events["mu1_eta"])>1.8) & (abs(events["mu1_eta"]) <=2.4)) & (abs(events["mu2_eta"])<=0.9))
    EO = (((abs(events["mu1_eta"])>1.8) & (abs(events["mu1_eta"]) <=2.4)) & ((abs(events["mu2_eta"])>0.9) & (abs(events["mu2_eta"]) <=1.8)))
    EE = (((abs(events["mu1_eta"])>1.8) & (abs(events["mu1_eta"]) <=2.4)) & ((abs(events["mu2_eta"])>1.8) & (abs(events["mu2_eta"]) <=2.4)))
    categories = [((events["mu1_pt"]>30)&(events["mu1_pt"]<=45)&(BB | OB | EB)),
                          ((events["mu1_pt"]>30)&(events["mu1_pt"]<=45)&(BO | OO | EO)),
                          ((events["mu1_pt"]>30)&(events["mu1_pt"]<=45)&(BE | OE | EE)),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&BB),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&BO),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&BE),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&OB),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&OO),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&OE),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&EB),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&EO),
                          ((events["mu1_pt"]>45)&(events["mu1_pt"]<=52)&EE),
                  # voigtian start here onwards
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&BB),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&BO),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&BE),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&OB),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&OO),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&OE),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&EB),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&EO),
                          ((events["mu1_pt"]>52)&(events["mu1_pt"]<=62)&EE),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&BB),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&BO),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&BE),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&OB),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&OO),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&OE),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&EB),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&EO),
                          ((events["mu1_pt"]>62)&(events["mu1_pt"]<=200)&EE),]
    return categories

In [7]:

# fields_of_interest = ["mu1_pt", "mu1_eta", "mu2_eta","dimuon_mass", "weights", "fraction"]
# # events = mc_events[fields_of_interest].compute() # eager version
# events = data_events[fields_of_interest]
# events

In [8]:
# """
# Eager load variable all at once Test
# """
# import time
# time_step = time.time()
# binning = np.linspace(*[70, 110, 100])
# # fraction_weight = ak.to_numpy(1/events.fraction.compute())
# fields_of_interest = ["mu1_pt", "mu1_eta", "mu2_eta","dimuon_mass", "weights", "fraction"]
# data_events = data_events[fields_of_interest].compute() # get eager version
# mc_events = mc_events[fields_of_interest].compute() # get eager version
# data_categories = get_calib_categories(data_events)
# mc_categories = get_calib_categories(mc_events)
# for idx in range(len(data_categories)):
#     cat_selection = data_categories[idx]
#     # print(f"cat_selection: {cat_selection.compute()}")
#     fraction_weight = 1/data_events.fraction
#     weights = data_events.weights*fraction_weight*cat_selection
#     # weights = ak.to_numpy(weights.compute())
#     dimuon_mass = ak.fill_none(data_events.dimuon_mass, value=-999.0)
#     # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
#     filter = (dimuon_mass!=-999.0) & (weights != 0)
#     dimuon_mass = dimuon_mass[filter]
#     weights = weights[filter]
#     data_hist, _ = np.histogram(dimuon_mass, bins=binning, weights = weights)
#     # MC ------------------------------------------------------------------------------------
#     cat_selection = mc_categories[idx]
#     # print(f"cat_selection: {cat_selection.compute()}")
#     fraction_weight = 1/mc_events.fraction
#     weights = mc_events.weights*fraction_weight*cat_selection
#     # weights = ak.to_numpy(weights.compute())
#     dimuon_mass = ak.fill_none(mc_events.dimuon_mass, value=-999.0)
#     # dimuon_mass = ak.to_numpy(dimuon_mass.compute())
#     filter = (dimuon_mass!=-999.0) & (weights != 0)
#     dimuon_mass = dimuon_mass[filter]
#     weights = weights[filter]
#     mc_hist, _ = np.histogram(dimuon_mass, bins=binning, weights = weights)
    

#     fig, ax_main = plt.subplots()
#     hep.histplot(
#         data_hist, xerr=True, 
#         # yerr=data_err,
#         bins=binning, stack=False, histtype='errorbar', color='black', 
#         label='Data', ax=ax_main
#     )
#     hep.histplot(mc_hist, bins=binning, 
#                              stack=False, histtype='fill', 
#                              label="MC", 
#                              # sort='label_r', 
#                              color="Green", 
#                              ax=ax_main)
#     ax_main.legend(loc="upper right")
#     ax_main.set_xlabel("dimuon mass [GeV]")
#     ax_main.set_title(f"Cat {idx} ZCR mass distribution")
#     plt.savefig(f"./cat{idx}.pdf")
#     ax_main.set_yscale('log')
#     ax_main.set_ylim(0.01, 1e9)
#     plt.savefig(f"./cat{idx}_log.pdf")
#     print(f"time elapsed in : {time.time()-time_step}")
#     time_step = time.time()
#     plt.clf()



In [9]:
"""
Eager load variable all at once Zip Test
"""
import time
time_step = time.time()
binning = np.linspace(*[70, 110, 100])
# fraction_weight = ak.to_numpy(1/events.fraction.compute())
fields_of_interest = ["mu1_pt", "mu1_eta", "mu2_eta","dimuon_mass", "weights", "fraction"]
data_events = data_events[fields_of_interest]
mc_events = mc_events[fields_of_interest]
dy_events = dy_events[fields_of_interest]
# zipping prior to computing adds slight runtime acceleration
data_events = ak.zip({
    field : data_events[field] for field in data_events.fields
}).compute()
mc_events = ak.zip({
    field : mc_events[field] for field in mc_events.fields
}).compute()
dy_events = ak.zip({
    field : dy_events[field] for field in dy_events.fields
}).compute()


data_categories = get_calib_categories(data_events)
mc_categories = get_calib_categories(mc_events)
dy_categories = get_calib_categories(dy_events)
for idx in range(len(data_categories)):
    # Data ----------------------------------------------------------
    cat_selection = data_categories[idx]
    fraction_weight = 1/data_events.fraction
    weights = data_events.weights*fraction_weight*cat_selection
    dimuon_mass = ak.fill_none(data_events.dimuon_mass, value=-999.0)
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    data_hist, _ = np.histogram(dimuon_mass, bins=binning, weights = weights)
    # MC ------------------------------------------------------------------------------------
    cat_selection = mc_categories[idx]
    weights = mc_events.weights*cat_selection
    dimuon_mass = ak.fill_none(mc_events.dimuon_mass, value=-999.0)
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    mc_hist, _ = np.histogram(dimuon_mass, bins=binning, weights = weights)

    # DY ------------------------------------------------------------------------------------
    cat_selection = dy_categories[idx]
    weights = dy_events.weights*cat_selection
    dimuon_mass = ak.fill_none(dy_events.dimuon_mass, value=-999.0)
    filter = (dimuon_mass!=-999.0) & (weights != 0)
    dimuon_mass = dimuon_mass[filter]
    weights = weights[filter]
    dy_hist, _ = np.histogram(dimuon_mass, bins=binning, weights = weights)
    

    fig, ax_main = plt.subplots()
    hep.histplot(
        data_hist, xerr=True, 
        # yerr=data_err,
        bins=binning, stack=False, histtype='errorbar', color='black', 
        label='Data', ax=ax_main
    )
    # hep.histplot(mc_hist, bins=binning, 
    #                          stack=False, histtype='fill', 
    #                          label="MC", 
    #                          # sort='label_r', 
    #                          color="Green", 
    #                          ax=ax_main)
    hep.histplot(
        [mc_hist, dy_hist],
        bins=binning, 
        stack=True, histtype='fill', 
        label=["Rest of MC","DY"], 
        # sort='label_r', 
        color=["Green","Orange"], 
        ax=ax_main
    )
    ax_main.legend(loc="upper right")
    ax_main.set_xlabel("dimuon mass [GeV]")
    ax_main.set_title(f"Cat {idx} ZCR mass distribution")
    plt.savefig(f"./cat{idx}.pdf")
    ax_main.set_yscale('log')
    ax_main.set_ylim(0.01, 1e9)
    plt.savefig(f"./cat{idx}_log.pdf")
    print(f"time elapsed in : {time.time()-time_step}")
    time_step = time.time()
    plt.close()
    plt.clf()



time elapsed in : 279.03707671165466
time elapsed in : 9.58542013168335
time elapsed in : 8.984301805496216
time elapsed in : 9.20556116104126
time elapsed in : 8.503501653671265
time elapsed in : 7.897006511688232
time elapsed in : 8.551406860351562
time elapsed in : 8.975012302398682
time elapsed in : 8.12041687965393
time elapsed in : 7.961039304733276
time elapsed in : 8.167770624160767
time elapsed in : 8.173918962478638
time elapsed in : 8.54919147491455
time elapsed in : 8.218966722488403
time elapsed in : 7.754816293716431
time elapsed in : 8.217082977294922
time elapsed in : 8.426958560943604
time elapsed in : 7.955994367599487
time elapsed in : 7.783867835998535
time elapsed in : 7.952259063720703
time elapsed in : 8.015288352966309
time elapsed in : 8.620718479156494
time elapsed in : 8.262346267700195
time elapsed in : 7.833375930786133
time elapsed in : 8.316704034805298
time elapsed in : 8.162906646728516
time elapsed in : 8.397667169570923
time elapsed in : 7.83771181106

<Figure size 1000x1000 with 0 Axes>